In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os


df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])

if not os.path.exists('results'):
    os.makedirs('results')

# Gráfico de caja
fig4 = px.box(
    df, 
    x="CATEGORIA", 
    y="PRECIO_UNITARIO_LOG", 
    color="CATEGORIA", 
    labels={"PRECIO_UNITARIO_LOG": "Precio Unitario (log)", "CATEGORIA": "Categoría"},
    width=900, height=500,
    template="plotly_white"
)

fig4.update_layout(
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=120)
)

fig4.update_yaxes(range=[0, df['PRECIO_UNITARIO_LOG'].max() + 1])
fig4.update_layout(xaxis={'categoryorder':'total descending'})

fig4.show()
fig4.write_image("results/Salud/boxplot_salud.pdf")

# Gráfico de volatilidad por categoría y fuente
vol_categoria_fuente = df.groupby(['CATEGORIA', 'FUENTE'])['PRECIO_UNITARIO_LOG'].std().reset_index()
vol_categoria_fuente = vol_categoria_fuente.rename(columns={'PRECIO_UNITARIO_LOG': 'VOLATILIDAD'})

fig_vol = go.Figure(data=go.Heatmap(
    z=vol_categoria_fuente['VOLATILIDAD'],
    x=vol_categoria_fuente['FUENTE'],
    y=vol_categoria_fuente['CATEGORIA'],
    colorscale='Teal',  
    colorbar=dict(title='Volatilidad'),
    showscale=True
))

for i, row in vol_categoria_fuente.iterrows():
    fig_vol.add_trace(go.Scatter(
        x=[row['FUENTE']], y=[row['CATEGORIA']],
        text=[f"{row['VOLATILIDAD']:.2f}"],
        mode="text",
        showlegend=False
    ))

fig_vol.update_layout(
    title=None,  
    yaxis_title="Categoría",
    margin=dict(l=40, r=40, t=40, b=120),
    width=900, height=500,
)

fig_vol.show()
fig_vol.write_image("results/Salud/heatmap_volatilidad_salud.pdf")

# Gráfico de línea de evolución del precio
df['FECHA'] = pd.to_datetime(df['FECHA'])
evolucion_precio = df.groupby(['FECHA', 'CATEGORIA'])['PRECIO_UNITARIO_LOG'].mean().reset_index()

fig_line = px.line(
    evolucion_precio, 
    x='FECHA', 
    y='PRECIO_UNITARIO_LOG',
    color='CATEGORIA', 
    markers=True,
    labels={'PRECIO_UNITARIO_LOG': 'Precio Unitario (log)', 'FECHA': 'Fecha', 'CATEGORIA': 'Categoría'},
    width=900, height=500,  
    template="plotly_white"
)

fig_line.update_layout(
    title=None, 
    xaxis_title="Fecha", 
    yaxis_title="Precio Unitario (log)", 
    margin=dict(l=40, r=40, t=40, b=120)
)

fig_line.show()
fig_line.write_image("results/Salud/evolucion_precio_salud.pdf")

# Gráfico de barras para el Coeficiente de Variación (CV)
cv_categoria_fuente = df.groupby(['CATEGORIA', 'FUENTE'])['PRECIO_UNITARIO_LOG'].agg(['mean', 'std']).reset_index()
cv_categoria_fuente['CV'] = cv_categoria_fuente['std'] / cv_categoria_fuente['mean'] * 100
cv_categoria_fuente = cv_categoria_fuente.rename(columns={'mean': 'MEDIA', 'std': 'DESVIACION'})

fig_cv = px.bar(
    cv_categoria_fuente, 
    x='CATEGORIA', 
    y='CV', 
    color='FUENTE', 
    labels={'CV': 'Coeficiente de Variación (%)', 'CATEGORIA': 'Categoría'},
    title=None,
    width=900, height=500,
    template="plotly_white",
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig_cv.update_layout(
    xaxis_title="Categoría", 
    yaxis_title="Coeficiente de Variación (%)", 
    margin=dict(l=40, r=40, t=40, b=120),
    legend_title="Fuente",  
    legend=dict(
        orientation="v",       
        yanchor="top",
        y=0.5,  
        xanchor="left",
        x=1.05  
    )
)

fig_cv.show()
fig_cv.write_image("results/Salud/cof_variacion_salud.pdf")
